# ЛАБОРАТОРНА РОБОТА  

## "Алгоритми та моделі перетворення даних"

__Метою__ лабораторної роботи є набуття практичних навичок використання модулів бібліотеки `Scikit-learn` для вирішення наступних задач:

- визначення та розподілення данних по їх типам
- вибір та застосування алгоритмів перетворення різноманітних типів даних
- настроювання параметрів та застосування трансформерів
- побудова конвеєра для автоматизації процесів трансформації даних

__Результатом__ виконання лабораторної роботи повинен стати датасет, в якому колонки з покажчиком перетворені під для навчання моделей

In [2]:
# імпортувати бібліотеки pandas та numpy
import pandas as pd
import numpy as np

## Завдання

Вхідним датасетом є csv-файл, отриманий в результаті виконання лабораторної роботи по конструюванню та очищеню покажчиків даних про продажі квартир рієлторської компанії.

<h4 style='background-color:cyan; color:red; padding:15px'><ul>
    <li>замість `...` вписуйте Ваш програмний код
    <li>не змінюйте імена запропонованих зміних
    </ul>
</h4>

In [3]:
# ззчитати підготовлений датасет `apartments_clean.csv` в датафрейм та проінспектувати його стан
data = pd.read_csv("apartments_clean.csv")
data.head()

,Price,rooms,price_per_m2,level,levels,year,area_total,area_living,area_kitchen,street,type,district,area_comfort
0,30970.0,1.0,1106.0,12.0,13.0,2013.0,28.0,13.0,5.0,Петропавлівська,площа,Оболонський,10.0
1,30970.0,1.0,1106.0,12.0,13.0,2013.0,28.0,13.0,5.0,Петропавлівська,вулиця,Подільський,10.0
2,82000.0,2.0,1281.0,18.0,26.0,2009.0,64.0,37.0,14.0,Дніпровська,вулиця,Оболонський,13.0
3,135000.0,2.0,2077.0,16.0,23.0,2012.0,65.0,25.0,18.0,Дніпровська,вулиця,Оболонський,22.0
4,84000.0,2.0,1273.0,6.0,26.0,2009.0,66.0,37.0,18.0,Дніпровська,вулиця,Оболонський,11.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Price         483 non-null    float64
 1   rooms         483 non-null    float64
 2   price_per_m2  483 non-null    float64
 3   level         483 non-null    float64
 4   levels        483 non-null    float64
 5   year          483 non-null    float64
 6   area_total    483 non-null    float64
 7   area_living   483 non-null    float64
 8   area_kitchen  483 non-null    float64
 9   street        483 non-null    object 
 10  type          483 non-null    object 
 11  district      483 non-null    object 
 12  area_comfort  483 non-null    float64
dtypes: float64(10), object(3)
memory usage: 49.2+ KB


## 1. Визначення типів покажчиків вхідного датасету

In [6]:
# відокремити від робочого датасету колонку з цільовим показчиком (`Price`) 
# та вилучити її з обробки
target = data['Price']
data.drop(columns='Price', inplace=True)
data

,rooms,price_per_m2,level,levels,year,area_total,area_living,area_kitchen,street,type,district,area_comfort
0,1.0,1106.0,12.0,13.0,2013.0,28.0,13.0,5.0,Петропавлівська,площа,Оболонський,10.0
1,1.0,1106.0,12.0,13.0,2013.0,28.0,13.0,5.0,Петропавлівська,вулиця,Подільський,10.0
2,2.0,1281.0,18.0,26.0,2009.0,64.0,37.0,14.0,Дніпровська,вулиця,Оболонський,13.0
3,2.0,2077.0,16.0,23.0,2012.0,65.0,25.0,18.0,Дніпровська,вулиця,Оболонський,22.0
4,2.0,1273.0,6.0,26.0,2009.0,66.0,37.0,18.0,Дніпровська,вулиця,Оболонський,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
478,2.0,998.0,1.0,5.0,2009.0,43.0,29.0,14.0,Іскрівська,вулиця,Солом'янський,0.0
479,1.0,2614.0,2.0,7.0,2019.0,44.0,14.0,21.0,Теремківська,вулиця,Голосіївський,9.0
480,1.0,1147.0,3.0,5.0,1960.0,34.0,18.0,8.0,Автозаводська,вулиця,Оболонський,8.0
481,3.0,1135.0,13.0,25.0,2004.0,69.0,37.0,14.0,Здолбунівська,вулиця,Дарницький,18.0


In [7]:
# для самоперевірки: кількисть рядків в датасетах повинна співпадати!
target.shape, data.shape

((483,), (483, 12))

In [8]:
# імпортувати з sklearn модуль, що відповідає за селективний відбір покажчиків
from sklearn.compose import make_column_selector as col_selector

In [9]:
# побудувати  селектори для вібіру числових та категоріальних колонок
numeric_selector = col_selector(dtype_include=np.number)
category_selector = col_selector(dtype_include=object)

In [10]:
# застосувати побудовані селектори для створення списків числових та категоріальних показчиків
numeric_data_columns = numeric_selector(data)
category_data_columns = category_selector(data)

print(f'список числових покажчиків = {numeric_data_columns}')
print(f'список категоріальних покажчиків = {category_data_columns}')

список числових покажчиків = ['rooms', 'price_per_m2', 'level', 'levels', 'year', 'area_total', 'area_living', 'area_kitchen', 'area_comfort']
список категоріальних покажчиків = ['street', 'type', 'district']


In [11]:
# створити 2 окремі датасети: числовий та категоріальний та вивести їх основні властивості
data_numeric = data[numeric_data_columns]
data_category = data[category_data_columns]

data_numeric.shape, data_category.shape

((483, 9), (483, 3))

## 2. Маштабування числових та категоріальних колонок

### 2.1. Маштабування числових показчиків

In [ ]:
# з модуля `preprocessing` імпортувати нормалізатор 'MinMaxScaler' та стандартизатор 'StandardScaler'
...

In [ ]:
# створити відповідні маштабувачі
standart_scaler = ...
min_max_scaler = ...

In [ ]:
# побудувати змаштабовані датасети за двома стратегіями: стандартизація та нормалізація
data_numeric_std = ...
data_numeric_min_max= ...

In [2]:
# побудувати графіки розподілення значень показчиків при
# стандартном та нормалізованом маштабуванні
...

#####  __проаналізувати графіки та визначись який маштабувач краще підходить до числовї підмножини робочого датасету__

### 2.1. Маштабування категоріальних показчиків

In [ ]:
# з модуля `preporcessing` імпортувати кодувальники 'OrdinalEncoder' та 'OneHotEncoder'
...

In [ ]:
# проаналізувати категоріальні показчики та визначитись до якого показчика
# який енкодер доцільно застосовувати
...

In [ ]:
# створити енкодери та закодувати ними відповідні категоріальні колонки
ordinal_encoder = ...
one_hot_encoder = ...

## 3. Побудова трансформера для підготовки даних

In [ ]:
# з модуля `compose` імпортувати трансформер колонок `ColumnTransformer`
...

In [ ]:
# побудувати трансформер для предобробки числових та категоріальних показчиків
# робочого датасету
transformer = ...
print(transformer)

In [ ]:
# отримати вихідний масив застосував трансфомер до вхідного датасуту
data_transformed = ...
data_transformed.shape

In [ ]:
# побудувати вихідний датафрейм з вихідного масиву додавши в першу колонку
# ціловий показчик `target`
apartment_transformed = ...

In [ ]:
# Проінспектувати вихідний датасет `apartment_transformed`
...

In [ ]:
# зберегти підготовлений датасет в csv-файлі `apartment_transformed.csv`
...